In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras import activations, initializers
from tensorflow.keras.layers import Layer

import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfp.__version__

'0.8.0'

In [2]:
#gives tl;dr
def bnn_extractor(flat_observations, net_arch, act_fun):
    """
    Constructs an variational layer that receives observations as an input and outputs a latent representation for the policy and
    a value network. The ``net_arch`` parameter allows to specify the amount and size of the hidden layers and how many
    of them are shared between the policy network and the value network. It is assumed to be a list with the following
    structure:
    1. An arbitrary length (zero allowed) number of integers each specifying the number of units in a shared layer.
       If the number of ints is zero, there will be no shared layers.
    2. An optional dict, to specify the following non-shared layers for the value network and the policy network.
       It is formatted like ``dict(vf=[<value layer sizes>], pi=[<policy layer sizes>])``.
       If it is missing any of the keys (pi or vf), no non-shared layers (empty list) is assumed.
    For example to construct a network with one shared layer of size 55 followed by two non-shared layers for the value
    network of size 255 and a single non-shared layer of size 128 for the policy network, the following layers_spec
    would be used: ``[55, dict(vf=[255, 255], pi=[128])]``. A simple shared network topology with two layers of size 128
    would be specified as [128, 128].
    :param flat_observations: (tf.Tensor) The observations to base policy and value function on.
    :param net_arch: ([int or dict]) The specification of the policy and value networks.
        See above for details on its formatting.
    :param act_fun: (tf function) The activation function to use for the networks.
    :return: (tf.Tensor, tf.Tensor) latent_policy, latent_value of the specified network.
        If all layers are shared, then ``latent_policy == latent_value``
    """
    
    latent = flat_observations
    policy_only_layers = []  # Layer sizes of the network that only belongs to the policy network
    value_only_layers = []  # Layer sizes of the network that only belongs to the value network
    kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p)

    # Iterate through the shared layers and build the shared parts of the network
    for idx, layer in enumerate(net_arch):
        if isinstance(layer, int):  # Check that this is a shared layer
            layer_size = layer
#             latent = act_fun(linear(latent, "shared_fc{}".format(idx), layer_size, init_scale=np.sqrt(2)))
            latent = act_fun(tfp.layers.DenseFlipout(layer_size, activation = 'relu', kernel_divergence_fn=kernel_divergence_fn)(latent))
        else:
            assert isinstance(layer, dict), "Error: the net_arch list can only contain ints and dicts"
            if 'pi' in layer:
                assert isinstance(layer['pi'], list), "Error: net_arch[-1]['pi'] must contain a list of integers."
                policy_only_layers = layer['pi']

            if 'vf' in layer:
                assert isinstance(layer['vf'], list), "Error: net_arch[-1]['vf'] must contain a list of integers."
                value_only_layers = layer['vf']
            break  # From here on the network splits up in policy and value network

    # Build the non-shared part of the network
    latent_policy = latent
    latent_value = latent
    for idx, (pi_layer_size, vf_layer_size) in enumerate(zip_longest(policy_only_layers, value_only_layers)):
        if pi_layer_size is not None:
            assert isinstance(pi_layer_size, int), "Error: net_arch[-1]['pi'] must only contain integers."
#             latent_policy = act_fun(linear(latent_policy, "pi_fc{}".format(idx), pi_layer_size, init_scale=np.sqrt(2)))
            latent_policy = act_fun(tfp.layers.DenseFlipout(pi_layer_size, activation = 'relu', kernel_divergence_fn=kernel_divergence_fn)(latent))

        if vf_layer_size is not None:
            assert isinstance(vf_layer_size, int), "Error: net_arch[-1]['vf'] must only contain integers."
#             latent_value = act_fun(linear(latent_value, "vf_fc{}".format(idx), vf_layer_size, init_scale=np.sqrt(2)))
            latent_value = act_fun(tfp.layers.DenseFlipout(vf_layer_size, activation = 'relu', kernel_divergence_fn=kernel_divergence_fn)(latent))

    return latent_policy, latent_value

In [3]:
from stable_baselines.common.policies import ActorCriticPolicy, nature_cnn

#something like an evaluation of the method used
class FeedForwardPolicy(ActorCriticPolicy):
    """
    Policy object that implements actor critic, using a feed forward neural network.
    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param layers: ([int]) (deprecated, use net_arch instead) The size of the Neural network for the policy
        (if None, default to [64, 64])
    :param net_arch: (list) Specification of the actor-critic policy network architecture (see mlp_extractor
        documentation for details).
    :param act_fun: (tf.func) the activation function to use in the neural network.
    :param cnn_extractor: (function (TensorFlow Tensor, ``**kwargs``): (TensorFlow Tensor)) the CNN feature extraction
    :param feature_extraction: (str) The feature extraction type ("cnn" or "mlp")
    :param kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, layers=None, net_arch=None,
                 act_fun=tf.tanh, cnn_extractor=nature_cnn, feature_extraction="cnn", **kwargs):
        super(FeedForwardPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=reuse,
                                                scale=(feature_extraction == "cnn"))

        self._kwargs_check(feature_extraction, kwargs)
        
        if layers is not None:
            warnings.warn("Usage of the `layers` parameter is deprecated! Use net_arch instead "
                          "(it has a different semantics though).", DeprecationWarning)
            if net_arch is not None:
                warnings.warn("The new `net_arch` parameter overrides the deprecated `layers` parameter!",
                              DeprecationWarning)

        if net_arch is None:
            if layers is None:
                layers = [64, 64]
            net_arch = [dict(vf=layers, pi=layers)]

        with tf.variable_scope("model", reuse=reuse):
            if feature_extraction == "cnn":
                pi_latent = vf_latent = cnn_extractor(self.processed_obs, **kwargs)
            elif feature_extraction == "bnn":
                pi_latent, vf_latent = bnn_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)
            else:
                pi_latent, vf_latent = mlp_extractor(tf.layers.flatten(self.processed_obs), net_arch, act_fun)

            self._value_fn = linear(vf_latent, 'vf', 1)

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
import warnings
from itertools import zip_longest
from abc import ABC, abstractmethod

import numpy as np
import tensorflow as tf
from gym.spaces import Discrete

from stable_baselines.common.tf_util import batch_to_seq, seq_to_batch
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc, lstm
from stable_baselines.common.distributions import make_proba_dist_type, CategoricalProbabilityDistribution, \
    MultiCategoricalProbabilityDistribution, DiagGaussianProbabilityDistribution, BernoulliProbabilityDistribution
from stable_baselines.common.input import observation_input
from stable_baselines.common.policies import nature_cnn

In [5]:
class BnnPolicy(FeedForwardPolicy):
    """
    Policy object that implements actor critic, using a Bayesian neural net (2 layers of 64)
    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param _kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **_kwargs):
        super(BnnPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse,
                                        feature_extraction="bnn", **_kwargs)

# DNN Cartpole

In [6]:
import os
import gym

from stable_baselines.ppo1 import PPO1
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import logger
from stable_baselines.common.callbacks import EvalCallback

NUM_TIMESTEPS = int(1e4)
SEED = 722
EVAL_FREQ = 250000
EVAL_EPISODES = 10  # was 1000

LOGDIR = "dnn_cartpole" # moved to zoo afterwards.
logger.configure(folder=LOGDIR)

env = gym.make("CartPole-v0")
env.seed(SEED)

Logging to dnn_cartpole


[722]

In [7]:
# take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
dnn = PPO1(MlpPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

dnn.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

dnn.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

env.close()





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
********** Iteration 0 ************


/opt/anaconda3/envs/try-bayesian/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x13b810650>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00974 |       0.00000 |      86.01871 |       0.00416 |       0.68911
     -0.01660 |       0.00000 |      72.62526 |       0.00789 |       0.68547
     -0.01867 |       0.00000 |      45.76273 |       0.00835 |       0.68501
     -0.02130 |       0.00000 |      22.93425 |       0.00919 |       0.68418
     -0.02393 |       0.00000 |      15.95167 |       0.01034 |       0.68305
     -0.02520 |       0.00000 |      14.53490 |       0.01165 |       0.68177
     -0.02676 |       0.00000 |      13.44226 |       0.01212 |       0.68130
     -0.02778 |       0.00000 |      12.26323 |       0.01270 |       0.68073
     -0.02821 |       0.00000 |      11.25233 |       0.01310 |       0.68034
     -0.02869 |       0.00000 |      10.50572 |       0.01317 |       0.68027
Evaluating losses...
     -0.02855 |       0.00000 |      10.21981 |       0.01142 |       0.68197
-----------------------------

# BNN Cartpole

In [7]:
NUM_TIMESTEPS = int(1e4)
SEED = 722
EVAL_FREQ = 250000
EVAL_EPISODES = 10  # was 1000

LOGDIR = "bnn_cartpole" # moved to zoo afterwards.
logger.configure(folder=LOGDIR)

env = gym.make("CartPole-v0")
env.seed(SEED)

Logging to bnn_cartpole


[722]

In [8]:
# take mujoco hyperparams (but doubled timesteps_per_actorbatch to cover more steps.)
bnn = PPO1(BnnPolicy, env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0, optim_epochs=10,
                 optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99, lam=0.95, schedule='linear', verbose=2)

eval_callback = EvalCallback(env, best_model_save_path=LOGDIR, log_path=LOGDIR, eval_freq=EVAL_FREQ, n_eval_episodes=EVAL_EPISODES)

bnn.learn(total_timesteps=NUM_TIMESTEPS, callback=eval_callback)

bnn.save(os.path.join(LOGDIR, "final_model")) # probably never get to this point.

env.close()





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
********** Iteration 0 ************


/opt/anaconda3/envs/try-bayesian/lib/python3.7/site-packages/stable_baselines/common/callbacks.py:287: UserWarning: Training and eval env are not of the same type<TimeLimit<CartPoleEnv<CartPole-v0>>> != <stable_baselines.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x1039b3dd0>
  "{} != {}".format(self.training_env, self.eval_env))


Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00148 |       0.00000 |      95.33205 |       0.00016 |       0.69298
     -0.00584 |       0.00000 |      92.44428 |       0.00108 |       0.69207
     -0.01160 |       0.00000 |      89.07041 |       0.00414 |       0.68904
     -0.01473 |       0.00000 |      84.93721 |       0.00732 |       0.68593
     -0.01566 |       0.00000 |      80.09994 |       0.00860 |       0.68469
     -0.01641 |       0.00000 |      74.67774 |       0.00837 |       0.68490
     -0.01734 |       0.00000 |      68.80899 |       0.00852 |       0.68475
     -0.01873 |       0.00000 |      62.70371 |       0.00899 |       0.68429
     -0.01968 |       0.00000 |      56.63303 |       0.00940 |       0.68388
     -0.02112 |       0.00000 |      50.76483 |       0.00978 |       0.68350
Evaluating losses...
     -0.02155 |       0.00000 |      47.86052 |       0.00897 |       0.68429
-----------------------------

In [9]:
import random

obs = env.reset()
done = False
total_reward = 0
while not done:
    if random.random() < 0.3:
        obs = np.random.rand(4,)
    action, _states = dnn.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    env.render()
env.close()
print("dnn score:", total_reward)
#exit()

dnn score: 49.0


In [9]:
import random

obs = env.reset()
done = False
total_reward = 0

while not done:
    if random.random() < 0.3:
        obs = np.random.rand(4,)
    action, _states = bnn.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    env.render()
env.close()
print("bnn score:", total_reward)
#exit()

bnn score: 18.0


# Performance

In [12]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=100)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=100)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

NameError: name 'dnn' is not defined

In [ ]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

In [ ]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

In [ ]:
from stable_baselines.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(dnn, dnn.get_env(), n_eval_episodes=1000)
print(f"DNN - Mean reward: {mean_reward}, Std reward: {std_reward}")

mean_reward, std_reward = evaluate_policy(bnn, bnn.get_env(), n_eval_episodes=1000)
print(f"BNN - Mean reward: {mean_reward}, Std reward: {std_reward}")